In [ ]:
import warnings

warnings.filterwarnings("ignore")

In [ ]:
from os.path import join
import torch

config = {
    "task": "multiclass",
    "data": {
        "name": "cifar-10",
        "batch_size": 32,
        "size": 32,
    },
    "network": {"input_channels": 1, "linear_size": 704, "num_classes": 10},
    "train": {
        "criterion": torch.nn.CrossEntropyLoss(),
        "checkpoint_save_dir": "checkpoints",
        "epochs": 200,
        "device": "cuda" if torch.cuda.is_available() else "cpu",
        "lr": 0.01,
    },
}

In [ ]:
TASK = config["task"]
SIZE = config["data"]["size"]
LR = config["train"]["lr"]
NUM_CLASSES = config["network"]["num_classes"]
EPOCHS = config["train"]["epochs"]
BATCH_SIZE = config["data"]["batch_size"]
INPUT_CHANNEL = config["network"]["input_channels"]
LINEAR_SIZE = config["network"]["linear_size"]
CRITERION = config["train"]["criterion"]
DEVICE = config["train"]["device"]
experiment_name = config["data"]["name"]

In [ ]:
checkpoint_save_dir = join(config["train"]["checkpoint_save_dir"], experiment_name)

In [ ]:
from torchvision import transforms
from torchvision import datasets
import torch
from torch.utils.data import DataLoader

transform = transforms.Compose([transforms.ToTensor()])

train_dataset = datasets.CIFAR10(
    root="./data/cifar10", train=True, download=True, transform=transform
)
test_dataset = datasets.CIFAR10(
    root="./data/cifar10", train=False, download=True, transform=transform
)
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)

In [ ]:
from resnet import get_resnet18_models

model_dict = get_resnet18_models(NUM_CLASSES)
optimizers_dict = {}
for model_name, model in model_dict.items():
    optimizers_dict[model_name] = torch.optim.Adam(model.parameters(), lr=LR)

In [ ]:
history_dict = {}

In [ ]:
from tcnn.utils.experiment.train import train_and_test_model

for model_name, model in model_dict.items():
    print(f"Training model {model_name}")
    history_dict[model_name] = train_and_test_model(
        model,
        train_loader,
        test_loader,
        CRITERION,
        optimizers_dict[model_name],
        scheduler=None,
        epochs=EPOCHS,
        save_checkpoint=True,
        save_checkpoint_interval=1,
        checkpoint_save_dir=join(checkpoint_save_dir, model_name),
        task=TASK,
    )
    print("***" * 10)

In [ ]:
from tcnn.utils.experiment.plot import plot_history

for model_name, history in history_dict.items():
    print(f"Model {model_name} history:")
    plot_history(history, model_name)

In [ ]:
import torch

input_shape = (BATCH_SIZE, 3, SIZE, SIZE)
input_tensor = torch.randn(input_shape).to(DEVICE)

In [ ]:
import torchprofile
from tcnn.utils.experiment.model import count_parameters
from tcnn.utils.experiment.train import eval_model

result = dict()
for model_name, model in model_dict.items():
    print(f"Evaluating model {model_name}")
    result[model_name] = dict()
    result[model_name]["macs"] = torchprofile.profile_macs(model, input_tensor)
    result[model_name]["params"] = count_parameters(model)
    result[model_name]["performance"] = eval_model(model, test_loader, CRITERION, TASK)
    with torch.no_grad():
        torch.cuda.empty_cache()
    print("***" * 10)

In [ ]:
from tcnn.utils.experiment.plot import plot_history_dict

plot_history_dict(history_dict)

In [ ]:
from tcnn.utils.experiment.log import show_test_result

show_test_result(result)